In [2]:
import os
import numpy as np
import pandas as pd
import torch
import random
import re

from tqdm import tqdm
from itertools import combinations
from collections import deque
from transformers import AutoTokenizer
# from transformers import AutoModel, AutoModelForSequenceClassification
# from datasets import load_dataset, load_metric
# from rank_bm25 import BM25Okapi
from sklearn.model_selection import train_test_split

BM25: 키워드 기반 랭킹 알고리즘
+ 주어진 쿼리에 대해 문서와의 연관성을 평가하는 랭킹 함수
+ Bag-of-words 개념을 사용하여 쿼리에 있는 용어가 각각의 문서에 얼마나 자주 등장하는지를 평가
    + 이때 IDF값을 곱해서 자주 등장하지 않는 단어에 더 큰 가중치를 줌.

In [3]:
def preprocess_script(script):
    new_script = deque()
    with open(script, 'r', encoding = 'utf-8') as file:
        lines = file.readlines()
        for line in lines:
            if line.lstrip().startswith('#'): # 주석으로 시작하는 행 skip
                continue
            line = line.rstrip()
            if '#' in line:
                line = line[:line.index('#')] # 주석 전까지 코드만 저장
            line = line.replace('\n', '') # 개행 문자를 모두 삭제
            line = line.replace('    ', '\t') # 공백 4칸을 tab으로 변환
            
            if line == '' : # 전처리 후 빈 라인은 skip
                continue
            
            new_script.append(line)
            
        new_script = '\n'.join(new_script) # 줄 마다 읽은 것을 개행 문자로 합침
        new_script = re.sub('("""[\w\W]*?""")', '<str>', new_script)
        new_script = re.sub("('''[\w\W]*?''')", '<str>', new_script)
        new_script = re.sub('/^(http?|https?):\/\/([a-z0-9-]+\.)+[a-z0-9]{2,4}.*$/', '', new_script)
        
    return new_script
        

In [4]:
def seed_everything(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  print(f"Seed set as {seed}")

seed_everything(42)

Seed set as 42


In [26]:
code_folder = './code'
problem_folders = os.listdir(code_folder) # code 안의 폴더 list를 반환

preprocess_scripts = []
problem_nums = []

# 300개 Sample code 에 대한 전처리
for problem_folder in tqdm(problem_folders):
    scripts = os.listdir(os.path.join(code_folder, problem_folder)) # code/problem000/.py 파일
    problem_num = problem_folder # 문제 번호 폴더명
    for script in scripts:
        script_file = os.path.join(code_folder, problem_folder, script)
        preprocessed_script = preprocess_script(script_file)
        
        preprocess_scripts.append(preprocessed_script)
    
    # 번호 목록을 만들어서 전처리한 dataframe 에 함께 넣어줌
    problem_nums.extend([problem_num]*len(scripts))

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [01:00<00:00,  4.96it/s]


In [27]:
len(preprocess_scripts), len(problem_nums)

(45101, 45101)

In [29]:
df = pd.DataFrame(data= {'code':preprocess_scripts, 'problem_num': problem_nums})
df

,code,problem_num
0,"M = 9\nN = 9\ndef main():\n\tfor i in range(1,...",problem001
1,"list1 = range(1,10)\nlist2 = list1\nfor i in l...",problem001
2,"for x in range(1, 10):\n\tfor y in range(1, 10...",problem001
3,"for i in range(1,10):\n\tfor n in range(1, 10)...",problem001
4,"for i in range(1,10):\n\tfor k in range(1,10):...",problem001
...,...,...
45096,def prime_factorize(n):\n\ta = [1]\n\twhile n ...,problem300
45097,"import math\nA,B = map(int,input().split())\ng...",problem300
45098,def prime_factorization(n):\n\ttable = []\n\tf...,problem300
45099,import fractions\nimport math\ndef primefac(n)...,problem300



### Create Data set

#### Tokenizer 수행, microsoft에서 개발한 사전 학습 모델인 graphcodebert 사용

In [31]:
# AutoTokenizer로 graphcodebert 사용
tokenizer = AutoTokenizer.from_pretrained('microsoft/graphcodebert-base')
tokenizer.truncation_side = 'left'
MAX_LEN = 512

In [32]:
tokens = []
for code in df['code']:
    tokens.append(tokenizer.tokenize(code, max_length = MAX_LEN, truncation = True))
    
df['tokens'] = tokens # Sample code를 Tokenization 해서 tokens 컬럼에 추가
df['len'] = df['tokens'].apply(len) # tokens의 길이를 len 컬럼에 추가

In [33]:
# train, valid set 분리
train_df, valid_df, train_label, valid_label = train_test_split(df, df['problem_num'], random_state=42, 
                                                                test_size = 0.1, stratify = df['problem_num'])

train_df = train_df.reset_index(drop=True) # Reindexing
valid_df = valid_df.reset_index(drop=True)

In [35]:
train_df.head()

,code,problem_num,tokens,len
0,"h,w,n = [int(input()) for i in range(3)]\nprin...",problem236,"[h, ,, w, ,, n, Ġ=, Ġ[, int, (, input, ()), Ġf...",34
1,"join,rate=map(int,input().split())\nif join>=1...",problem211,"[join, ,, rate, =, map, (, int, ,, input, ().,...",41
2,"n, k = map(int, input().split())\nh = sorted(l...",problem227,"[n, ,, Ġk, Ġ=, Ġmap, (, int, ,, Ġinput, ()., s...",46
3,"N = int(input())\na_list = list(map(int, input...",problem257,"[N, Ġ=, Ġint, (, input, ()), Ċ, a, _, list, Ġ=...",87
4,N=int(input())\nD=[{} for _ in range(N+1)]\nfo...,problem262,"[N, =, int, (, input, ()), Ċ, D, =[, {, }, Ġfo...",218


#### Create Level 1 dataset

In [38]:
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes 는 code 가 쭉 나열된 상태
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [41]:
total_positive_pairs = []
total_negative_pairs = []

In [66]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes 로 저장
    # 이때 train_df 에는 problem_num 이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = train_df[train_df['problem_num'] == problem]['code'].to_list()
    other_codes = train_df[train_df['problem_num'] != problem]['code'].to_list()
    
    # positive_pairs 1000개 (총 300 * 1000 = 300,000개)
    # negative_paris 1000개 (총 300 * 1000 = 300,000개)
    positive_pairs = list(combinations(solution_codes, 2))
    random.shuffle(positive_pairs)
    random.shuffle(other_codes)
    positive_pairs = positive_pairs[:1000]
    other_codes = other_codes[:1000]
    
    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))
        
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [01:04<00:00,  4.68it/s]


In [70]:
# total_positive_pairs 와 negative_pairs 의 정답 코드를 묶어 code1로 지정
# total_positive_pairs 와 negative_pairs 의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame 으로 선언
train_data = pd.DataFrame(data = {'code1' : code1, 'code2' : code2, 'similar' : label})
train_data = train_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
train_data.to_csv('train_data_lv1.csv', index = False)

#### Validation data set

In [71]:
codes = valid_df['code'].to_list() # code 컬럼을 list로 변환 - codes 는 code가 쭉 나열된 형태
problems = valid_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [72]:
total_positive_pairs = []
total_negative_pairs = []

In [73]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = valid_df[valid_df['problem_num'] == problem]['code'].to_list()
    other_codes = valid_df[valid_df['problem_num'] != problem]['code'].to_list()
    
    # positive_pairs 100개 (총 300 * 100 = 30,000개) 추출
    # negative_pairs 100개 (총 300 * 100 = 30,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:100]
    random.shuffle(other_codes)
    other_codes = other_codes[:100]
    
    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|██████████| 300/300 [00:07<00:00, 42.17it/s]


In [74]:
# total_positive_pairs 와 negative_pairs 의 정답 코드를 묶어 codes1로 지정
# total_positive_pairs 와 negative_pairs 의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
valid_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
valid_data = valid_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
valid_data.to_csv('valid_data_lv1.csv',index=False)